# Setup instructions

Before running this notebook:
* Add Azure credentials specific to your Azure account in the configuration.json file. The configuration file should be in the same directory as this Jupyter notebook.

# Connect

In [ ]:
from __future__ import print_function

import sys
from os import listdir
from os.path import isfile, join

import azure.mgmt.batchai.models as models
from azure.storage.blob import BlockBlobService
from azure.storage.file import FileService

import utilities as utils
from utilities.job_factory import ParameterSweep, NumericParameter, DiscreteParameter

cfg = utils.config.Configuration('configuration.json')
client = utils.config.create_batchai_client(cfg)

# Create resource group & batch AI workspace

In [ ]:
utils.config.create_resource_group(cfg)
_ = client.workspaces.create(cfg.resource_group, cfg.workspace, cfg.location).result()

# Create file share 

In [ ]:
# File share
azure_file_share_name = 'sat-solver'
file_service = FileService(cfg.storage_account_name, cfg.storage_account_key)
file_service.create_share(azure_file_share_name, fail_on_exist=False)

# Upload problem set

In [ ]:
local_dir = 'max3sat-problems'
azure_file_share_prefix_path = 'max3sat-problems'

# Create the directory path in the cluster file share if necessary.
if azure_file_share_prefix_path:
    file_service.create_directory(
        azure_file_share_name, azure_file_share_prefix_path, fail_on_exist=False)
    
def upload_all_files(directory):
    for filename in [f for f in listdir(directory) if isfile(join(directory, f))]:
        print(filename, end=' ')
        file_service.create_file_from_path(
            azure_file_share_name, azure_file_share_prefix_path, filename, local_dir + '/' + filename)
    print()

upload_all_files(local_dir)

# Upload algorithm

In [ ]:
local_dir = 'algorithm'
azure_file_share_prefix_path = ''

# Create the directory path in the cluster file share if necessary.
if azure_file_share_prefix_path:
    file_service.create_directory(
        azure_file_share_name, azure_file_share_prefix_path, fail_on_exist=False)

def upload_file(filename):
    file_service.create_file_from_path(
        azure_file_share_name, azure_file_share_prefix_path, filename, local_dir + '/' + filename)

upload_file('algorithm.tar.gz')

# Configure compute cluster

In [ ]:
nodes_count = 1
cluster_name = 'sat-solver'
vm_type = 'STANDARD_D1'
setup_output_prefix = '/node-setup-logs'

parameters = models.ClusterCreateParameters(
    vm_size=vm_type,
    scale_settings=models.ScaleSettings(
        manual=models.ManualScaleSettings(target_node_count=nodes_count)
    ),
    user_account_settings=models.UserAccountSettings(
        admin_user_name=cfg.admin,
        admin_user_password=cfg.admin_password or None,
        admin_user_ssh_public_key=cfg.admin_ssh_key or None,
    ),
    node_setup=models.NodeSetup(
        setup_task=models.SetupTask(
            command_line='apt -y install gcc make',
            std_out_err_path_prefix=setup_output_prefix
        )
    )
)

# Create Compute Cluster

In [ ]:
_ = client.clusters.create(cfg.resource_group, cfg.workspace, cluster_name, parameters).result()

# Monitor Cluster Creation

In [ ]:
cluster = client.clusters.get(cfg.resource_group, cfg.workspace, cluster_name)
utils.cluster.print_cluster_status(cluster)

# Create experiment

In [ ]:
experiment_name = 'sweep001'
experiment = client.experiments.create(cfg.resource_group, cfg.workspace, experiment_name).result()

# Define sweep paramteres

In [ ]:
# param_specs = [
#     NumericParameter(
#         parameter_name="N",
#         data_type="INTEGER",
#         start=1,
#         end=2,
#         step=2,
#         scale="LINEAR"
#     )
# ]

param_specs = [
    DiscreteParameter(
        parameter_name="DURATION",
        values=[5000, 20000, 40000]
    )
]

parameters = ParameterSweep(param_specs)

# Create job template

In [ ]:
azure_file_share_mount_path = 'afs'
relative_code_path = f'{azure_file_share_mount_path}/{azure_file_share_prefix_path}'
relative_data_path = f'{azure_file_share_mount_path}/max3sat-problems'


jcp = models.JobCreateParameters(
    cluster=models.ResourceId(id=cluster.id),
    
    node_count=1,
    
    std_out_err_path_prefix = f'$AZ_BATCHAI_JOB_MOUNT_ROOT/{azure_file_share_mount_path}',
    
    output_directories = [
        models.OutputDirectory(
            id='ALL',
            path_prefix=f'$AZ_BATCHAI_JOB_MOUNT_ROOT/{azure_file_share_mount_path}'
        )
    ],
    
    mount_volumes = models.MountVolumes(
        azure_file_shares=[
            models.AzureFileShareReference(
                account_name=cfg.storage_account_name,
                credentials=models.AzureStorageCredentialsInfo(
                    account_key=cfg.storage_account_key),
                azure_file_url=f'https://{cfg.storage_account_name}.file.core.windows.net/{azure_file_share_name}',
                relative_mount_path=azure_file_share_mount_path)
        ]
    ),
    
    custom_toolkit_settings=models.CustomToolkitSettings(
        command_line=f'tar xzf $AZ_BATCHAI_JOB_MOUNT_ROOT/{relative_code_path}/algorithm.tar.gz -C $AZ_BATCH_TASK_WORKING_DIR ; ./run.sh $AZ_BATCHAI_JOB_MOUNT_ROOT/{relative_data_path} v90c700 {parameters["DURATION"]}'
    )
    
    #, container_settings = 

) 

# Create jobs with specific parameters

In [ ]:
try:
    job_count = job_count + 1
except NameError:
    job_count = 1

job_prefix = f'job{job_count:03d}'

# Generate Jobs
jobs_to_submit, param_combinations = parameters.generate_jobs(jcp)

# Print the parameter combinations generated
for idx, comb in enumerate(param_combinations):
    print(f"Parameters {idx + 1}: {comb}")

# Submit Jobs
experiment_utils = utils.experiment.ExperimentUtils(client, cfg.resource_group, cfg.workspace, experiment_name)
jobs = experiment_utils.submit_jobs(jobs_to_submit, job_prefix).result()

# Extract metric

In [ ]:
metric_extractor = utils.job.MetricExtractor(
                        output_dir_id='stdouterr',
                        logfile='stdout.txt',
                        regex='metric = ([0-9]*)')


# Wait for all jobs to complete
experiment_utils.wait_all_jobs()

# Get the metrics from the jobs
results = experiment_utils.get_metrics_for_jobs(jobs, metric_extractor)

In [ ]:
# Print results
for result in results:
    print(result)